In [ ]:
#import packages and functions

%load_ext autoreload
%autoreload 2

import pursuit_functions as pursuit
    
import pandas as pd
import numpy as np
import polars as pl
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 
from numba import njit

from scipy import stats
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
#load data set

all_pursuit_tasks = pd.read_parquet("ca1_ca3_rsc_pursuit_data.parquet", engine="pyarrow")

# Normalize points and find circle boundaries.

In [ ]:
#get all coordinate values below 99th percentile and normalize points for all regions 

normalized_sessions = pursuit.tuning.normalize_points(all_pursuit_tasks)

In [ ]:
#find the mean center and overall radius of the arena for all normalized data points
#you can specify the percentile value to be considered for the overall radius; default is 95th percentile
#calculates the individual center point for each session

circle_boundaries, radius = pursuit.tuning.fit_circle_bounds(normalized_sessions)
print(radius)

In [ ]:
#find circumference points for plotting using the center coordinates and overall radius
all_circ_points = pursuit.tuning.circumference(circle_boundaries)

# Plot the laser coordinates and boundaries.

In [ ]:
#plot normalized concatenated laser and rat paths with center point and boundary
#the function takes the normalized_sessions, circle_boundaries, and all_circ_points dataframes

pursuit.tuning.plot_arena_bounds(normalized_sessions, circle_boundaries, all_circ_points)

# Clean data and pull spike data.

In [ ]:
# obtain region-specific sessions
RSC_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "RSC"]
CA1_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "CA1"]
CA3_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "CA3"]



In [ ]:
#obtain trial block-specific sessions

RSC_pursuit = RSC_sessions[RSC_sessions["trial_block"] == "pursuit"]
CA1_pursuit = CA1_sessions[CA1_sessions["trial_block"] == "pursuit"]
CA3_pursuit = CA3_sessions[CA3_sessions["trial_block"] == "pursuit"]

In [ ]:
#drop NA values for RSC, CA1, and CA3 sessions

RSC_cleaned = pursuit.tuning.drop_NA_vals(RSC_pursuit)
CA1_cleaned = pursuit.tuning.drop_NA_vals(CA1_pursuit)
CA3_cleaned = pursuit.tuning.drop_NA_vals(CA3_pursuit)

# Normalizing time for bootstrapping.

In [ ]:
# there's some data compression that collapses the 120hz recording time points as the recordings get longer. 
# first we collapse the time points to whole seconds
# we need to normalize the time so that the first observation starts from 0 seconds
# we then calculate the normalized minute each observation belongs to

def normalize_time(dataframe):
    df = dataframe.copy()
    df["time"] = df["time"].astype(float)
    df["relative_time"] = df.groupby("sessFile")["time"].transform(lambda x: x - x.min())
    df["norm_sec"] = df["relative_time"].astype(int)
    df["norm_min"] = df["norm_sec"] // 60

    return df


In [ ]:
RSC_clean_time = pursuit.tuning.normalize_time(RSC_cleaned)
CA1_clean_time = pursuit.tuning.normalize_time(CA1_cleaned)
CA3_clean_time = pursuit.tuning.normalize_time(CA3_cleaned)

# Assigning epochs to dataframes with normalized time.

In [ ]:
# to split each session into epochs, we separate them by first/second half of the recording or odd/even minutes
# for epoch_half, True:1, False:2 will result in the first half labeled as 1 (<= cutoff) and the second half labeled as 2
# for epoch_odd_even, we take the minutes divided by 2 and find the remainder. If the remainder is 1 (odd), it will be labeled as 1 and if the remainder is 0 (even), it will be labeled as 2

def assign_epochs(dataframe):
    df = dataframe.copy()

    # separate epochs by half

    def label_half(group):
        mins = group["norm_min"].unique()
        mins.sort()
        cutoff = mins[len(mins) // 2]
        return group["norm_min"] <= cutoff
    
    df["epoch_half"] = (
        df.groupby("sessFile", group_keys=False)
        .apply(label_half, include_groups=False)
        .map({True: 1, False: 2})
    )

    # separate epochs by odd/even minutes

    df["epoch_odd_even"] = df["norm_min"] % 2
    df["epoch_odd_even"] = df["epoch_odd_even"].map({1: 1, 0: 2})

    return df


In [ ]:
# for epoch_half, True:1, False:2 will result in the first half labeled as 1 (<= cutoff) and the second half labeled as 2
# for epoch_odd_even, we take the minutes divided by 2 and find the remainder. If the remainder is 1 (odd), it will be labeled as 1 and if the remainder is 0 (even), it will be labeled as 2

RSC_clean_time_epochs = pursuit.tuning.assign_epochs(RSC_clean_time)
CA1_clean_time_epochs = pursuit.tuning.assign_epochs(CA1_clean_time)
CA3_clean_time_epochs = pursuit.tuning.assign_epochs(CA3_clean_time)

# Create concise dataframes with only sessFile, laser, epoch, spike, and relative time data.

In [ ]:
#make a new df with only sessFile, laser, epoch, spike, and relative time data.
def epoch_laser_spks(dataframe, laser_x="laserPos_1", laser_y="laserPos_2"):

    epoch_laser_spks_data = []

    spk_columns = [col for col in dataframe.columns if "spkTable" in col]

    time_column = [col for col in dataframe.columns if "relative_time" in col.lower()]
    
    for sessFile in dataframe["sessFile"].unique():

            session = dataframe[dataframe["sessFile"] == sessFile].copy()
            
            laser_x_vals = session[laser_x].astype("float64")
            laser_y_vals = session[laser_y].astype("float64")
    
            #identify 99th percentile x, y boundaries
            x_low, x_high = np.percentile(laser_x_vals, [0, 99])
            y_low, y_high = np.percentile(laser_y_vals, [0, 99])

            #filter the data so we only get the data under the 99th percentile
            filter = (
                (laser_x_vals >= x_low) & (laser_x_vals <= x_high) & 
                (laser_y_vals >= y_low) & (laser_y_vals <= y_high)
            )

            filtered_session = session[filter].copy()

            #normalize the points to the origin
            x_normalized = filtered_session[laser_x].astype("float64") - float(x_low)
            y_normalized = filtered_session[laser_y].astype("float64") - float(y_low)

            #grab epoch data 
            epoch_half = filtered_session["epoch_half"].values
            epoch_odd_even = filtered_session["epoch_odd_even"].values

            #make a dataframe containing normalized data
            normalized_df = pd.DataFrame({
                "sessFile": sessFile,
                "laser_x_normalized": x_normalized.values,
                "laser_y_normalized": y_normalized.values,
                "epoch_half": epoch_half,
                "epoch_odd_even": epoch_odd_even
            })

            #grab spike data using the normalized data mask
            spk_df = filtered_session[spk_columns].reset_index(drop=True)

            #grab time data using the normalized data mask
            time_df = filtered_session[time_column].reset_index(drop=True)

            #make a combined dataframe
            combined_df = pd.concat([normalized_df.reset_index(drop=True), spk_df, time_df], axis=1)

            #append dataframe to the list
            epoch_laser_spks_data.append(combined_df)

        #make a giant dataframe by concatenating all the dataframes in the list        
    epoch_laser_spks_df = pd.concat(epoch_laser_spks_data, ignore_index=True)

    return epoch_laser_spks_df

In [ ]:
#make a new df with only sessFile, laser, epoch, spike, and relative time data.

RSC_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(RSC_clean_time_epochs)
CA1_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(CA1_clean_time_epochs)
CA3_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(CA3_clean_time_epochs)

# Pull epoch halves into two data frames for tuning correlation.

In [ ]:
#function for pulling epochs from each session along with associated sessFile, laser, and spike data

def pull_epochs(dataframe, 
                spk_prefix="spkTable"):

    epoch_first_half = []
    epoch_second_half = []
    epoch_odd_min = []
    epoch_even_min = []

    for sessFile in dataframe["sessFile"].unique():
        
        session = dataframe[dataframe["sessFile"] == sessFile].copy()

        # mapping first half and second epochs for each session 
        first_half = session[session["epoch_half"] == 1]
        second_half = session[session["epoch_half"] ==2]
        # mapping odd and even epochs for each session
        odd_min = session[session["epoch_odd_even"] == 1]
        even_min = session[session["epoch_odd_even"] == 2]

        spk_cols = [col for col in session.columns if spk_prefix in col and not session[col].isna().all()]

        #function for grabbing sessFile, laser x, laser y, and spk columns for each epoch
        def build_epoch_df(epoch):
            return pd.concat([
                epoch[["sessFile", "laser_x_normalized", "laser_y_normalized", "relative_time"]].reset_index(drop=True),
                epoch[spk_cols].reset_index(drop=True)
            ], axis=1)

        epoch_first_half.append(build_epoch_df(first_half))
        epoch_second_half.append(build_epoch_df(second_half))
        epoch_odd_min.append(build_epoch_df(odd_min))
        epoch_even_min.append(build_epoch_df(even_min))

    return (
        pd.concat(epoch_first_half, ignore_index=True),
        pd.concat(epoch_second_half, ignore_index=True),
        pd.concat(epoch_odd_min, ignore_index=True),
        pd.concat(epoch_even_min, ignore_index=True),
    )
    


In [ ]:
RSC_epoch_first_half, RSC_epoch_second_half, RSC_epoch_odd_min, RSC_epoch_even_min = pursuit.tuning.pull_epochs(RSC_epoch_laser_spks)
CA1_epoch_first_half, CA1_epoch_second_half, CA1_epoch_odd_min, CA1_epoch_even_min = pursuit.tuning.pull_epochs(CA1_epoch_laser_spks)
CA3_epoch_first_half, CA3_epoch_second_half, CA3_epoch_odd_min, CA3_epoch_even_min = pursuit.tuning.pull_epochs(CA3_epoch_laser_spks)

# Bootstrapping Functions.

In [ ]:
# boostrapping function calculates the true corr then uses the relative time to shift the spktables before calculating the distance, bin spks/laser vals, and calculating tuning

@njit
def make_assignment_matrix(bin_assignments, dist_bin_edges):
    bin_ids = np.digitize(bin_assignments, dist_bin_edges, right=False) - 1
    bin_mask = (bin_ids >= 0) & (bin_ids < len(dist_bin_edges) -1)

    B = len(dist_bin_edges) - 1
    T = len(bin_assignments)
    M = np.zeros((B, T))
    for t in range(T):
        if bin_mask[t]:
            M[bin_ids[t], t] += 1
    return M


In [ ]:
@njit
def tuning_calc(M, spk_array):
    spk_sum = M @ spk_array # (B, N)
    occupancy = M.sum(axis=1) # (B,)

    tuning = np.zeros_like(spk_sum)

    for b in range(spk_sum.shape[0]):
            if occupancy[b] != 0:
                for n in range(spk_sum.shape[1]):
                    tuning[b, n] = spk_sum[b, n] / occupancy[b]    
    
    return tuning, occupancy

In [ ]:
def make_shift_idx(rel_time, num_shifts=10):
    rel_time_np = rel_time.to_numpy().flatten()
    shift_points = np.where(np.diff(np.floor(rel_time_np)) > 0)[0] + 1
    return shift_points[:num_shifts].tolist()

In [ ]:
def process_session(epoch_df1, epoch_df2, center_df, sessFile, spk_cols, num_shifts, rel_time_col="relative_time"):
        
    all_results = []

    valid_spk_cols = [
        col for col in spk_cols
        if not (epoch_df1[col].isna().all() or epoch_df1[col].sum() == 0) and
            not (epoch_df2[col].isna().all() or epoch_df2[col].sum() == 0)
        ]

    # calculate distance to bounds
    dist1 = pursuit.tuning.dist_to_bounds(epoch_df1, center_df)["bound_dist"].values
    dist2 = pursuit.tuning.dist_to_bounds(epoch_df2, center_df)["bound_dist"].values

    # get spike arrays
    spk1 = epoch_df1[valid_spk_cols].to_numpy(dtype=np.float64)
    spk2 = epoch_df2[valid_spk_cols].to_numpy(dtype=np.float64)

    # get bin edges
    dist_bin_edges = np.linspace(min(dist1.min(), dist2.min()),
                                max(dist1.max(), dist2.max()), 21)                                   

    # make bin assignment matrices 
    M1 = make_assignment_matrix(dist1, dist_bin_edges)
    M2 = make_assignment_matrix(dist2, dist_bin_edges)

    # calculate tuning
    tune1, _ = tuning_calc(M1, spk1)
    tune2, _ = tuning_calc(M2, spk2)

    # spearman corr- iter: 0 (True corr)
    for j, ncol in enumerate(valid_spk_cols):
        r, p = stats.spearmanr(tune1[:, j], tune2[:, j])
        all_results.append({
            "sessFile": sessFile,
            "neuron": ncol,
            "bootstrap_iter": 0,
            "spearman_r": float(r),
            "p_val": float(p)
        })

    #bootstrapping with shift indices from rel_time

    rel_time = epoch_df1[rel_time_col]
    shift_points = make_shift_idx(rel_time, num_shifts=num_shifts)

    for i, shift_idx in enumerate(shift_points, start=1):
        spk1_shifted = np.roll(spk1, shift_idx, axis=0) 
        tune1_rolled, _ = tuning_calc(M1, spk1_shifted)

        for j, ncol in enumerate(valid_spk_cols):
            r, p = stats.spearmanr(tune1_rolled[:, j], tune2[:, j])
            all_results.append({
                "sessFile": sessFile,
                "neuron": ncol,
                "bootstrap_iter": i,
                "spearman_r": float(r),
                "p_val": float(p)
                })

    return all_results

In [ ]:
def bootstrap_all_sessions(epoch_df1, epoch_df2, center_df, spk_prefix="spkTable", rel_time_col="relative_time", num_shifts=1000):
    spk_cols = [col for col in epoch_df1.columns if spk_prefix in col]
    sessions = epoch_df1["sessFile"].unique()

    all_bootstrap_results = []

    for sessFile in tqdm(sessions, desc="Sessions"):
        df1 = epoch_df1[epoch_df1["sessFile"] == sessFile]
        df2 = epoch_df2[epoch_df2["sessFile"] == sessFile]

        results = process_session(df1, df2, center_df, sessFile=sessFile, spk_cols=spk_cols, num_shifts=num_shifts, rel_time_col=rel_time_col)
        all_bootstrap_results.extend(results)

    return pd.DataFrame(all_bootstrap_results)

In [ ]:
#bootstrap sessions
RSC_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(RSC_epoch_first_half, RSC_epoch_second_half, circle_boundaries)
RSC_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(RSC_epoch_odd_min, RSC_epoch_even_min, circle_boundaries)

CA1_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(CA1_epoch_first_half, CA1_epoch_second_half, circle_boundaries)
CA1_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(CA1_epoch_odd_min, CA1_epoch_even_min, circle_boundaries)

CA3_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(CA3_epoch_first_half, CA3_epoch_second_half, circle_boundaries)
CA3_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(CA3_epoch_odd_min, CA3_epoch_even_min, circle_boundaries)

In [ ]:
RSC_first_second_tuning

In [ ]:
RSC_odd_even_tuning

# Find cells with corr values over the 95th and 99th percentiles and compare them to the true tuning values.

In [ ]:
def plot_null_dist(df, neurons=None, percentile_lines=(95, 99), max_neurons=None):
    grouped = df.groupby(["sessFile", "neuron"])

    if neurons is None:
        neurons = list(grouped.groups.keys())
        if max_neurons is not None:
            neurons = neurons[:max_neurons]

    for sessFile, neuron in neurons:
        sub_df = grouped.get_group((sessFile, neuron))

        null_df = sub_df[sub_df["bootstrap_iter"] > 0]
        true_df = sub_df[sub_df["bootstrap_iter"] == 0]
        
        fig, ax = plt.subplots(figsize=(8,5))

        sns.rugplot(
            x=null_df["spearman_r"],
            height=0.05,
            hue=null_df["bootstrap_iter"],
            ax=ax,
            lw=0.05
        )

        sns.histplot(
            null_df["spearman_r"],
            bins = 30,
            stat="count",
            kde=False,
            color='lightblue',
            edgecolor='white',
            ax=ax,
            alpha=0.6
        )

        true_r = true_df["spearman_r"].values[0]
        ax.axvline(true_r, color='red', linestyle='--', label=f"True r ={true_r:.3f}")

        for p in percentile_lines:
            threshold = np.percentile(null_df["spearman_r"], p)
            ax.axvline(threshold, color='gray', linestyle=':', label=f"{p}th ={threshold:.3f}")

        ax.set_title(f"Null dist — {sessFile} / {neuron}")
        ax.set_xlabel("Spearman r")
        ax.set_ylabel("Bootstrap count")
        ax.legend()
        plt.tight_layout()
        plt.show()

In [ ]:
#plot null distribution of spearman r values with true, 95th, 99th percentile lines for all cells
pursuit.tuning.plot_null_dist(RSC_first_second_tuning)

In [ ]:
# we're going to get the significant cells now

def get_significant_cells(df, percentile=95):
    significant_cells = []

    grouped = df.groupby(["sessFile", "neuron"])

    for (sessFile, neuron), group in grouped:
        true_r = group.loc[group["bootstrap_iter"] == 0, "spearman_r"].values
        true_r = true_r[0]

        boot_r = group.loc[group["bootstrap_iter"] > 0, "spearman_r"]
        threshold = np.percentile(boot_r, percentile)

        if true_r > threshold:
            significant_cells.append((sessFile, neuron))

    return significant_cells    

In [ ]:
#ID significant cells above 95th and 99th corr values
RSC_first_second_cells_95 = pursuit.tuning.get_significant_cells(RSC_first_second_tuning, percentile=95)
RSC_first_second_cells_99 = pursuit.tuning.get_significant_cells(RSC_first_second_tuning, percentile=99)


In [ ]:
#plot null distribution of spearman r values with true, 95th, 99th percentile lines for significant cells
pursuit.tuning.plot_null_dist(RSC_first_second_tuning, neurons=RSC_first_second_cells_95)

In [ ]:
def sig_cells_heatmap(raw_data, center_df, sig_cells_list, percentile=95,
                                 smoothing_window=3, smoothing_std=1, plot_title=None):
    
    df_sig_cells = pd.DataFrame(sig_cells_list, columns=["sessFile", "neuron"])

   
    df_laser_spks = pursuit.tuning.norm_laser_get_spks(raw_data)

   
    df_bounds = pursuit.tuning.dist_to_bounds(df_laser_spks, center_df)

    
    df_binned = pursuit.tuning.bin_spikes_laser(df_bounds)

    
    df_tuning = pursuit.tuning.calculate_tuning(df_binned)

   
    df_tuning_filtered = df_tuning.merge(df_sig_cells, on=["sessFile", "neuron"])

   
    df_z = pursuit.tuning.z_score_norm(df_tuning_filtered)

   
    df_smoothed = pursuit.tuning.pivot_smooth(df_z, window_size=smoothing_window, std=smoothing_std)

   
    df_sorted = pursuit.tuning.peak_sort(df_smoothed)

   
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_sorted, cmap="viridis", annot=False, fmt=".2f", yticklabels=False)
    plt.title(plot_title or f"Z-scored Spike Activity (> {percentile}th percentile)")
    plt.xlabel("Boundary Distance (bin midpoint)")
    plt.ylabel("Neurons (peak sorted)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return df_sorted, df_binned  



In [ ]:
# plot heatmaps of significant cells and return two dataframes; one for peak-sorted neurons ready to plot and one with binned tuning values 
df_sorted , df_binned = pursuit.tuning.sig_cells_heatmap(RSC_cleaned, circle_boundaries, RSC_first_second_cells_95, percentile=95)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

def occ_dist_to_pdf(df, output_path = "RSC_sessions_laser_occ.pdf"):
    
    grouped = df.groupby(["sessFile", "bin_midpoint"], observed=True)["laser_occupancy"].first().reset_index()

    occ_stats = (
        grouped.groupby("bin_midpoint", observed=True)["laser_occupancy"]
        .agg(["mean"])
        .reset_index()
        .rename(columns={"mean": "mean_occ"})
    )

    merged = grouped.merge(occ_stats, on="bin_midpoint", how="left")

    with PdfPages(output_path) as pdf:

        for sessFile, group in merged.groupby("sessFile"):
            fig, ax = plt.subplots(figsize=(10, 6))

            sns.barplot(
                x=group["bin_midpoint"].astype(str),
                y=group["laser_occupancy"],
                ax=ax,
                color='lightblue',
                edgecolor='black',
                label="Session Laser Occupancy"
            )

            ax.errorbar(
                x=np.arange(len(group)),
                y=group["mean_occ"],
                fmt='o',
                color='black',
                capsize=4,
                label="Global Mean"
            )

            ax.set_title(f"Laser Occupancy by Bin - {sessFile}")
            ax.set_xlabel("Boundary Distance (bin midpoint)")
            ax.set_ylabel("Laser Occupancy")
            plt.xticks(rotation=45, ha="right")
            ax.legend()
            plt.tight_layout()
            #plt.show()
            
            pdf.savefig(fig)
            plt.close(fig)

In [ ]:
#plot at total laser occupancy per bin per session (and save to pdf bc there's a lot of plots)

pursuit.tuning.occ_dist_to_pdf(df_binned)

In [ ]:
def plot_sig_tuning_curves(raw_data, center_df, sig_cells_list, percentile=95,
                                 smoothing_window=3, smoothing_std=1, plot_title=None):
    
    df_sig_cells = pd.DataFrame(sig_cells_list, columns=["sessFile", "neuron"])

   
    df_laser_spks = pursuit.tuning.norm_laser_get_spks(raw_data)

   
    df_bounds = pursuit.tuning.dist_to_bounds(df_laser_spks, center_df)

    
    df_binned = pursuit.tuning.bin_spikes_laser(df_bounds)

    
    df_tuning = pursuit.tuning.calculate_tuning(df_binned)

    
    df_tuning_filtered = df_tuning.merge(df_sig_cells, on=["sessFile", "neuron"])


    grouped = df_tuning_filtered.groupby(["sessFile", "neuron"])

    for (sessFile, neuron), sub_df in grouped:
        
        pivoted = (sub_df.pivot(index="neuron", columns="bin_midpoint", values="tuning").fillna(0))

        for neuron in pivoted.index:
            plt.plot(pivoted.columns, pivoted.loc[neuron], marker='o', linestyle='-', label=f"{neuron}")

        plt.xlabel("Boundary Distance (bin midpoint)")
        plt.ylabel("Tuning (spike count / laser occupancy)")
        plt.title(f"Tuning Curve — {sessFile} / {neuron}")
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:
#plot individual significant cell tuning curves

pursuit.tuning.plot_sig_tuning_curves(RSC_cleaned, circle_boundaries, RSC_first_second_cells_95)

# ignore everything under here for now

# Investigating time shifts.

In [ ]:
def unique_time_entries(dataframe):
    all_sessions = []

    for sessFile in dataframe["sessFile"].unique():
        session = dataframe[dataframe["sessFile"] == sessFile]
        session_times = session["time"]

        times_unique = session_times.astype("float64").nunique()
        times_count = session_times.astype("float64").value_counts().to_dict()

        all_sessions.append({
            "sessFile": sessFile,
            "times_unique": times_unique,
            "unique_times_count": times_count
        })

    return pd.DataFrame(all_sessions)

In [ ]:
RSC_sessions_times = unique_time_entries(RSC_sessions)
RSC_sessions_times

In [ ]:
def inspect_time_counts(session_row):
    times= session_row["unique_times_count"]
    sorted_times = sorted(times.items())
    return pd.DataFrame(sorted_times, columns=["time", "count"])

In [ ]:
lp03_25 = RSC_sessions_times[RSC_sessions_times["sessFile"] == "LP03_25_pursuitRoot.mat"]

lp03_25_row = lp03_25.iloc[0]

lp03_25_inspection = inspect_time_counts(lp03_25_row)


lp03_25_inspection

# Find distance of laser points to boundary and bin data by distance.

In [ ]:
#find distance of normalized laser points to circle boundary by each session
#function takes the normalized laser/spikes and circle boundaries dataframes

RSC_laser_spks_bounds = pursuit.tuning.dist_to_bounds(RSC_laser_spks, circle_boundaries)
CA1_laser_spks_bounds = pursuit.tuning.dist_to_bounds(CA1_laser_spks, circle_boundaries)
CA3_laser_spks_bounds = pursuit.tuning.dist_to_bounds(CA3_laser_spks, circle_boundaries)

In [ ]:
CA3_laser_spks_bounds.head(50)

In [ ]:
#bin the data!
RSC_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(RSC_laser_spks_bounds)
CA1_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(CA1_laser_spks_bounds)
CA3_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(CA3_laser_spks_bounds)

# Normalize spike counts, smooth data, peak sort neurons, and plot tuning curves.

In [ ]:
#calculate raw tuning curves
RSC_tuning = pursuit.tuning.calculate_tuning(RSC_laser_spikes_binned)
CA1_tuning = pursuit.tuning.calculate_tuning(CA1_laser_spikes_binned)
CA3_tuning = pursuit.tuning.calculate_tuning(CA3_laser_spikes_binned)

In [ ]:
#plot all neuron tuning curves for a sanity check

pursuit.tuning.plot_tuning_curves(CA3_tuning)

In [ ]:
# z-score and normalize data
RSC_z_scored = pursuit.tuning.z_score_norm(RSC_tuning)
CA1_z_scored = pursuit.tuning.z_score_norm(CA1_tuning)
CA3_z_scored = pursuit.tuning.z_score_norm(CA3_tuning)

In [ ]:
# apply smoothing and pivot data
RSC_smoothed = pursuit.tuning.pivot_smooth(RSC_z_scored)
CA1_smoothed = pursuit.tuning.pivot_smooth(CA1_z_scored)
CA3_smoothed = pursuit.tuning.pivot_smooth(CA3_z_scored)

In [ ]:
# peak sort the data
RSC_smoothed_sorted = pursuit.tuning.peak_sort(RSC_smoothed)
CA1_smoothed_sorted = pursuit.tuning.peak_sort(CA1_smoothed)
CA3_smoothed_sorted = pursuit.tuning.peak_sort(CA3_smoothed)

In [ ]:
#plot heatmaps
pursuit.tuning.heatmap(RSC_smoothed_sorted)
pursuit.tuning.heatmap(CA1_smoothed_sorted)
pursuit.tuning.heatmap(CA3_smoothed_sorted)

# Calculate neuron counts

In [ ]:
RSC_count = pursuit.tuning.count_neurons(RSC_cleaned)
CA1_count = pursuit.tuning.count_neurons(CA1_cleaned)
CA3_count = pursuit.tuning.count_neurons(CA3_cleaned)

In [ ]:
RSC_count


In [ ]:
CA1_count

In [ ]:
CA3_count